## Imports and settings

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import json
import pprint
import collections

import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import ipywidgets as widgets
from matplotlib import gridspec
import pprint
from scipy.stats import gaussian_kde
from tqdm import tqdm

detector_path = '..'
results_path = os.path.join(detector_path, 'results')
sys.path.append(detector_path)

from utils import constants
from utils import errors
from sleep.mass import MASS
from sleep.mass_k import MASSK
from sleep.inta import INTA
from sleep import postprocessing
from sleep import data_ops
from evaluation import metrics
from evaluation import data_manipulation

SEED = 123
DPI = 200

%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Load database and predicted probabilities

In [ ]:
# Select database
dataset_name = constants.MASSK_NAME
# Select predictions ckpt folder

# Best ones:
# MASS: ckpt_folder = os.path.join('20190413_bsf_ss_using_angle_train_mass', 'bsf', 'avg'), thr 0.45
# INTA: ckpt_folder = os.path.join('20190328_v2bn_fixed_inta_train_inta', 'bsf_avg'), thr 0.48
# MASSK: ckpt_folder = os.path.join('20190413_bsf_kc_using_angle_train_massk', 'bsf', 'avg'), thr 0.48

# ckpt_folder = os.path.join('20190414_bsf_kc_whole_night_train_massk', 'seed0')
ckpt_folder = os.path.join('20190413_bsf_kc_using_angle_train_massk', 'bsf', 'avg')
optimal_thr = 0.48
whole_night = False
verbose = False

# Load data

# Marks are binary sequences for each page, 200 fs resolution
errors.check_valid_value(
    dataset_name, 'dataset_name',
    [constants.MASS_NAME, constants.INTA_NAME, constants.MASSK_NAME])
if dataset_name == constants.MASS_NAME:
    dataset = MASS(load_checkpoint=True)
elif dataset_name == constants.INTA_NAME:
    dataset = INTA(load_checkpoint=True)
else:
    dataset = MASSK(load_checkpoint=True)
    
    # Get training set ids
print('Loading train set and splitting train/val')
all_train_ids = dataset.train_ids
# Split to form validation set
train_ids, val_ids = data_manipulation.split_ids_list(
    all_train_ids, seed=SEED)
print('Training set IDs:', train_ids)
print('Validation set IDs:', val_ids)
# Get test data
print('Loading test set')
test_ids = dataset.test_ids
print('Testing set IDs:', test_ids)
idx_dict = {'train': train_ids, 'val': val_ids, 'test': test_ids, 'alltrain': all_train_ids}

# Get subjects data, with the expert used for training
print('Loading signals and marks')
# set_list = ['train', 'val', 'test']
set_list = ['alltrain', 'test']

x = {}
y = {}
pages = {}
x['alltrain'], y['alltrain'] = dataset.get_subset_data(all_train_ids, which_expert=1, verbose=verbose, whole_night=whole_night)
x['train'], y['train'] = dataset.get_subset_data(train_ids, which_expert=1, verbose=verbose, whole_night=whole_night)
x['val'], y['val'] = dataset.get_subset_data(val_ids, which_expert=1, verbose=verbose, whole_night=whole_night)
x['test'], y['test'] = dataset.get_subset_data(test_ids, which_expert=1, verbose=verbose, whole_night=whole_night)
print('Loading pages')
pages['alltrain'] = dataset.get_subset_pages(all_train_ids, verbose=verbose, whole_night=whole_night)
pages['train'] = dataset.get_subset_pages(train_ids, verbose=verbose, whole_night=whole_night)
pages['val'] = dataset.get_subset_pages(val_ids, whole_night=whole_night)
pages['test'] = dataset.get_subset_pages(test_ids, whole_night=whole_night)

# Load predictions (probability vectors for each page), 200/factor resolution (default factor 8)
ckpt_path = os.path.abspath(os.path.join(results_path, 'predictions_%s' % dataset_name, ckpt_folder))
print('Loading predictions from %s' % ckpt_path)
y_pred = {}

if whole_night:
    descriptor = '_whole_night_'
else:
    descriptor = '_'

for set_name in set_list:
    y_pred[set_name] = np.load(os.path.join(ckpt_path, 'y_pred%s%s.npy' % (descriptor, set_name)), allow_pickle=True)
    # Keep only class 1 probability
    y_pred[set_name] = [this_y_pred[..., 1] for this_y_pred in y_pred[set_name]]
print('Done')

if dataset_name == constants.MASSK_NAME:
    # KC
    min_separation = 0
    min_duration = 0.3
    max_duration = 4.0
else:  
    # SS
    min_separation = 0.3
    min_duration = 0.2
    max_duration = 4.0

In [ ]:
# Prepare expert labels into marks
print('Preparing labels... ', end='', flush=True)
y_stamps = {}
for set_name in set_list:
    y_stamps[set_name] = postprocessing.generate_mark_intervals_with_list(
        y[set_name], pages[set_name], 200, 200, thr=None, postprocess=False)
print('Done')

In [ ]:
# Load second expert in mass
y2 = {}
if dataset_name == constants.MASS_NAME:
    _, y2['train'] = dataset.get_subset_data(train_ids, which_expert=2, verbose=verbose)
    _, y2['val'] = dataset.get_subset_data(val_ids, which_expert=2, verbose=verbose)
    _, y2['test'] = dataset.get_subset_data(test_ids, which_expert=2, verbose=verbose)
print('Preparing labels of E2... ', end='', flush=True)
y2_stamps = {}
for set_name in set_list:
    y2_stamps[set_name] = postprocessing.generate_mark_intervals_with_list(
        y2[set_name], pages[set_name], 200, 200, thr=None, postprocess=False)
print('Done')

## Cheating performance: Looking for Individualized thresholds

In [ ]:
# Performance settings
res_thr = 0.02
start_thr = 0.1
end_thr = 0.9
n_thr = int(np.round((end_thr - start_thr) / res_thr + 1))
thr_list = [start_thr + res_thr * i for i in range(n_thr)]
# print(thr_list)
print('Number of thresholds to be evaluated: %d' % len(thr_list))

# ---------------- Compute performance
af1 = {}
# Go through several IoU values
first_iou = 0
last_iou = 1
res_iou = 0.01
n_points = int(np.round((last_iou - first_iou) / res_iou))
full_iou_list = np.arange(n_points + 1) * res_iou + first_iou

for set_name in set_list:
    af1[set_name] = []
    n_subjects = len(y_pred[set_name])
    for _ in range(n_subjects):
        af1[set_name].append([])
    my_it = tqdm(thr_list)
    for thr in my_it:
        my_it.set_description('Processing %s set, threshold %1.3f' % (set_name, thr))
        # Prepare model predictions
        # print('Preparing predictions', flush=True)
        y_pred_thr = postprocessing.generate_mark_intervals_with_list(
            y_pred[set_name], pages[set_name], 
            fs_input = 200//8, fs_output=200, thr=thr, postprocess=True,
            min_separation=min_separation, min_duration=min_duration,
            max_duration=max_duration)
        # Go through several IoU values
        # print('Computing F1 Curve... ', flush=True, end='')
        all_f1_list = [metrics.f1_vs_iou(this_y, this_y_pred, full_iou_list) 
                       for (this_y, this_y_pred) 
                       in zip(y_stamps[set_name], y_pred_thr)]
        all_af1 = [this_result.mean() for this_result in all_f1_list]
        for j in range(n_subjects):
            af1[set_name][j].append(all_af1[j])

# Find optimal thresholds for each subjects
optimal_thr_dict = {}
for set_name in set_list:
    print('\nSubset %s' % set_name)
    optimal_thr_dict[set_name] = []
    n_subjects = len(y_pred[set_name])
    for j in range(n_subjects):
        subject_idx = idx_dict[set_name][j]
        max_idx = np.argmax(af1[set_name][j])
        this_optimal = thr_list[max_idx]
        optimal_thr_dict[set_name].append(this_optimal)
        print('Optimal for S%02d: %1.4f' % (subject_idx, optimal_thr_dict[set_name][j]))

## Performance: F1 vs IoU curve

In [ ]:
# Performance settings
chosen_set = 'test'
thr = optimal_thr_dict

# ---------------- Compute performance

# Prepare expert labels into marks
y_thr = y_stamps[chosen_set]

# Prepare model predictions
print('Preparing predictions', flush=True)
n_subjects = len(y_thr)
if isinstance(thr, collections.Mapping):
    print('Using thr dictionary and %s set' % (chosen_set))
    y_pred_thr = []
    for j in range(n_subjects):
        this_pred_thr = postprocessing.generate_mark_intervals(
            y_pred[chosen_set][j], pages[chosen_set][j], 200//8, 200, thr=thr[chosen_set][j],
            min_separation=min_separation, min_duration=min_duration, max_duration=max_duration)
        y_pred_thr.append(this_pred_thr)
else:
    print('Using thr %1.4f and %s set' % (thr, chosen_set))
    y_pred_thr = postprocessing.generate_mark_intervals_with_list(
        y_pred[chosen_set], pages[chosen_set], 200//8, 200, thr=thr,
        min_separation=min_separation, min_duration=min_duration, max_duration=max_duration)


# Go through several IoU values
print('Computing F1 Curve', flush=True)
first_iou = 0
last_iou = 1
res_iou = 0.01
n_points = int(np.round((last_iou - first_iou) / res_iou))
full_iou_list = np.arange(n_points + 1) * res_iou + first_iou
short_iou_list = np.arange(11) * 0.1

# Full
all_f1_list = [metrics.f1_vs_iou(this_y, this_y_pred, full_iou_list) 
               for (this_y, this_y_pred) 
               in zip(y_thr, y_pred_thr)]
all_f1_list = np.stack(all_f1_list, axis=1)
mean_f1 = np.mean(all_f1_list, axis=1)
std_f1 = np.std(all_f1_list, axis=1)
model_f1_mean = np.stack([full_iou_list, mean_f1], axis=1)
model_f1_std = np.stack([full_iou_list, std_f1], axis=1)

# Short
all_f1_list = [metrics.f1_vs_iou(this_y, this_y_pred, short_iou_list) 
               for (this_y, this_y_pred) 
               in zip(y_thr, y_pred_thr)]
all_f1_list = np.stack(all_f1_list, axis=1)
mean_f1 = np.mean(all_f1_list, axis=1)
std_f1 = np.std(all_f1_list, axis=1)
short_model_f1_mean = np.stack([short_iou_list, mean_f1], axis=1)
short_model_f1_std = np.stack([short_iou_list, std_f1], axis=1)


print('Mean F1')
pprint.pprint(short_model_f1_mean)
print('Std F1')
pprint.pprint(short_model_f1_std)
print('%s AF1: %1.4f' % (chosen_set.capitalize(), model_f1_mean[:, 1].mean()))

In [ ]:
# Single metric: average F1 on validation set
chosen_set_af1 = 'alltrain'
validation_af1 = metrics.average_f1_with_list(
    y[chosen_set_af1], y_pred[chosen_set_af1], pages[chosen_set_af1], thr=thr, 
    min_separation=min_separation, min_duration=min_duration, max_duration=max_duration)
print('%s AF1: %1.4f' % (chosen_set_af1.capitalize(), validation_af1))

In [ ]:
# Saving settings
save_f1_iou_result = False
ckpt_id = ''

# Comparison settings
comparison_folder = 'comparison_data'
compare_expert = True
compare_chambon = True
show_set_std = False
alpha = 0.2
color_list = {'model': '#c62828', 'expert': '#455a64', 'chambon': '#0277bd'} 
zoom_xlim = [0.1, 0.7]
zoom_ylim = [0.6, 0.85]
linewidth_model = 2
markersize_model = 10
linewidth_others = 1.3
markersize_others = 6

# --------------- Optional: Save F1 data
if save_f1_iou_result:
    filename = os.path.join('comparison_data', '%s_f1_vs_iou_model_%s.csv' % (dataset_name, ckpt_id))
    np.savetxt(filename, model_f1_mean, delimiter=",")


# -------------------- P L O T ----------------------    
# Comparison data
if compare_expert and (dataset_name == constants.MASS_NAME or dataset_name == constants.INTA_NAME):
    expert_f1_curve_mean = np.loadtxt(os.path.join(comparison_folder, 'f1_vs_iou_expert_mean.csv'), delimiter=',')
    expert_f1_curve_std = np.loadtxt(os.path.join(comparison_folder, 'f1_vs_iou_expert_std.csv'), delimiter=',')
if compare_chambon and dataset_name == constants.MASS_NAME:
    # chambon_f1_curve_1 = np.loadtxt(os.path.join(comparison_folder, 'mass_f1_vs_iou_Chambon_jointly.csv'), delimiter=',')
    chambon_f1_curve_2 = np.loadtxt(os.path.join(comparison_folder, 'mass_f1_vs_iou_Chambon_separately.csv'), delimiter=',')
    # chambon_f1_curve = np.stack([chambon_f1_curve_1, chambon_f1_curve_2], axis=2).max(axis=2)
    chambon_f1_curve = chambon_f1_curve_2
if compare_chambon and dataset_name == constants.MASSK_NAME:
    # chambon_f1_curve_1 = np.loadtxt(os.path.join(comparison_folder, 'massk_f1_vs_iou_Chambon_jointly.csv'), delimiter=',')
    chambon_f1_curve_2 = np.loadtxt(os.path.join(comparison_folder, 'massk_f1_vs_iou_Chambon_separately.csv'), delimiter=',')
    # chambon_f1_curve = np.stack([chambon_f1_curve_1, chambon_f1_curve_2], axis=2).max(axis=2)
    chambon_f1_curve = chambon_f1_curve_2

fig, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=DPI)

# Complete plot
if compare_expert and (dataset_name == constants.MASS_NAME or dataset_name == constants.INTA_NAME):
    ax.plot(expert_f1_curve_mean[:, 0], expert_f1_curve_mean[:, 1], linewidth=linewidth_others, 
               markersize=markersize_others, marker='.', 
               label='Expert Performance\nPrivate Database\nWarby et al. 2014', color=color_list['expert'])
    ax.fill_between(
        expert_f1_curve_mean[:, 0], 
        expert_f1_curve_mean[:, 1] - expert_f1_curve_std[:, 1], 
        expert_f1_curve_mean[:, 1] + expert_f1_curve_std[:, 1], 
        alpha=alpha, facecolor=color_list['expert'])
if compare_chambon and (dataset_name == constants.MASS_NAME or dataset_name == constants.MASSK_NAME):
    ax.plot(chambon_f1_curve[:, 0], chambon_f1_curve[:, 1], linewidth=linewidth_others, 
               markersize=markersize_others, marker='.', 
               label='DOSED (ConvNet)\nChambon et al. 2018', color=color_list['chambon'])
ax.plot(model_f1_mean[:, 0], model_f1_mean[:, 1], 
           linewidth=linewidth_model, # markersize=markersize_model, marker='.', 
           label='Proposed Model\n%s Database' % dataset_name.upper(), color=color_list['model'])
ax.plot(short_model_f1_mean[:, 0], short_model_f1_mean[:, 1], 
           linestyle='none', markersize=markersize_model, marker='.', color=color_list['model'])
if show_set_std:
    ax.fill_between(
            model_f1_mean[:, 0], 
            model_f1_mean[:, 1] - model_f1_std[:, 1], 
            model_f1_mean[:, 1] + model_f1_std[:, 1], 
            alpha=alpha, facecolor=color_list['model'])
if isinstance(thr, collections.Mapping):
    ax.set_title('Performance with individualized $\mu$ (%s)' % (chosen_set.capitalize()), fontsize=11)
else:
    ax.set_title('Performance with $\mu=%1.3f$ (%s)' % (thr, chosen_set.capitalize()), fontsize=11)
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])
ax.set_yticks([0.1*i for i in range(1, 10)])
ax.set_xticks([0.1*i for i in range(1, 10)])
ax.tick_params(labelsize=8.5)
ax.set_xlabel('IoU Threshold', fontsize=8.5)
ax.set_ylabel('F1-score', fontsize=8.5)
ax.grid()

ax.legend(loc='lower left', bbox_to_anchor=(1.05, 0.15), labelspacing=3, fontsize=7)

plt.show()

## Performance: Precision-Recall plot, separated subjects

In [ ]:
thr = optimal_thr_dict

iou_thr = 0.3

# Prepare model predictions
be_stats = {}
for set_name in set_list:
    print('Preparing predictions for %s set' % set_name, flush=True) 
    n_subjects = len(y_pred[set_name])
    if isinstance(thr, collections.Mapping):
        print('Using thr dictionary and %s set' % (chosen_set))
        y_pred_thr = []
        for j in range(n_subjects):
            this_pred_thr = postprocessing.generate_mark_intervals(
                y_pred[set_name][j], pages[set_name][j], 200//8, 200, thr=thr[set_name][j],
                min_separation=min_separation, min_duration=min_duration, max_duration=max_duration)
            y_pred_thr.append(this_pred_thr)
    else:
        print('Using thr %1.4f and %s set' % (thr, chosen_set))
        y_pred_thr = postprocessing.generate_mark_intervals_with_list(
            y_pred[set_name], pages[set_name], 200//8, 200, thr=thr,
            min_separation=min_separation, min_duration=min_duration, max_duration=max_duration)    
    be_stats[set_name] = [metrics.by_event_confusion(this_y, this_y_pred, iou_thr=iou_thr) 
                for (this_y, this_y_pred) in zip(y_stamps[set_name], y_pred_thr)]
    print('Done')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=DPI)
text_space = 0.01
markersize = 20

compare_chambon = True

if compare_chambon and dataset_name == constants.MASS_NAME:
    chambon_f1_curve_joint = np.loadtxt(os.path.join(comparison_folder, 'mass_pr_Chambon_jointly.csv'), delimiter=',')
    chambon_f1_curve_sepa = np.loadtxt(os.path.join(comparison_folder, 'mass_pr_Chambon_separately.csv'), delimiter=',')
if compare_chambon and dataset_name == constants.MASSK_NAME:
    chambon_f1_curve_joint = np.loadtxt(os.path.join(comparison_folder, 'massk_pr_Chambon_jointly.csv'), delimiter=',')
    chambon_f1_curve_sepa = np.loadtxt(os.path.join(comparison_folder, 'massk_pr_Chambon_separately.csv'), delimiter=',')

# F1 score levels
delta = 0.01 
x_ = np.arange(1, 100) * delta 
y_ = np.arange(1, 100) * delta 
X, Y = np.meshgrid(x_, y_)
Z = 2 * X * Y / (X + Y)
CS = ax.contour(X, Y, Z, colors='k', alpha=0.3, levels=[0.6, 0.7, 0.8, 0.9])
ax.clabel(CS, fontsize=7.5, fmt='%1.2f')

# Scatter plots of each subject
color_list = {'train':'#43a047', 'val':'#0288d1', 'test': '#c62828', 'alltrain':'#43a047'}
for set_name in set_list:
    for i, stats in enumerate(be_stats[set_name]):
        if i==0:
            ax.scatter(stats['recall'], stats['precision'], c=color_list[set_name], label='Proposed (%s)' % set_name.capitalize(), s=markersize, zorder=10)
        else:
            ax.scatter(stats['recall'], stats['precision'], c=color_list[set_name], s=markersize, zorder=10)
        ax.annotate(idx_dict[set_name][i], (stats['recall']+text_space, stats['precision']+text_space), fontsize=7, color='#1b2631', zorder=20)  
if compare_chambon and (dataset_name == constants.MASSK_NAME or dataset_name == constants.MASS_NAME):
    ax.scatter(chambon_f1_curve_joint[0], chambon_f1_curve_joint[1], 
               c='#455a64', label='DOSED Jointly', s=markersize, zorder=10, marker='s')
    ax.scatter(chambon_f1_curve_sepa[0], chambon_f1_curve_sepa[1], 
               c='#455a64', label='DOSED Separately', s=markersize, zorder=10, marker='o')
if isinstance(thr, collections.Mapping):
    ax.set_title('Test PR with individualized $\mu$ and IoU$>$%1.1f (%s)' % (iou_thr, dataset_name.upper()), fontsize=10)
else:
    ax.set_title('Test PR with $\mu=%1.3f$ and IoU$>$%1.1f (%s)' % (thr, iou_thr, dataset_name.upper()), fontsize=10)
        

ax.set_xlabel('Recall', fontsize=8.5)
ax.set_ylabel('Precision', fontsize=8.5)
ax.set_xlim([0.4, 1])
ax.set_ylim([0.4, 1])
ax.legend(loc='lower left', fontsize=6.5)
ax.tick_params(labelsize=8.5)
ax.grid()
plt.show()

## Performance: Precision-Recall curve, separated subjects

In [ ]:
# The mean of each set is drawn, for several thr

iou_thr = 0.3
this_thr = optimal_thr_dict

res_thr = 0.02
start_thr = 0.1
end_thr = 0.9
n_thr = int(np.round((end_thr - start_thr) / res_thr))
thr_list = np.arange(n_thr+1) * res_thr + start_thr

# Prepare model predictions
pr_curve = {}
n_thr = len(thr_list)
for set_name in set_list:
    n_subjects = len(y_pred[set_name])
    pr_curve[set_name] = []
    for _ in range(n_subjects):
        pr_curve[set_name].append(np.zeros((n_thr, 2)))   # Columns are [x: recall, y: precision]
    my_it = tqdm(enumerate(thr_list))
    for i, thr in my_it:
        my_it.set_description('Processing %s set, threshold %1.2f' % (set_name, thr))
        y_pred_thr = postprocessing.generate_mark_intervals_with_list(
            y_pred[set_name], pages[set_name], 200//8, 200, thr=thr,
            min_separation=min_separation, min_duration=min_duration, max_duration=max_duration
        )
        
        this_stats = [metrics.by_event_confusion(this_y, this_y_pred, iou_thr=iou_thr) 
                     for (this_y, this_y_pred) in zip(y_stamps[set_name], y_pred_thr)]
        for j in range(n_subjects):
            pr_curve[set_name][j][i, 0] = this_stats[j]['recall']
            pr_curve[set_name][j][i, 1] = this_stats[j]['precision']

In [ ]:
# Plot settings
show_train = True
show_val = True

thr_to_show = this_thr

subjects_to_show = None

# ---------------------------------------------------------
fig, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=DPI)
markersize = 30
text_space = 0.01
alpha=0.8

# F1 score levels
delta = 0.01 
x_ = np.arange(1, 100) * delta 
y_ = np.arange(1, 100) * delta 
X, Y = np.meshgrid(x_, y_)
Z = 2 * X * Y / (X + Y)
CS = ax.contour(X, Y, Z, colors='k', alpha=0.3, levels=[0.6, 0.7, 0.8, 0.9])
ax.clabel(CS, fontsize=7.5, fmt='%1.2f')

set_list_aux = []
if show_train and ('alltrain' in set_list):
    set_list_aux.append('alltrain')
if show_train and ('train' in set_list):
    set_list_aux.append('train')
if show_val and ('val' in set_list):
    set_list_aux.append('val')
set_list_aux.append('test')

# Scatter plots of each subset

color_list = {'train':'#43a047', 'val':'#0288d1', 'test': '#c62828', 'alltrain':'#43a047'}
for i, set_name in enumerate(set_list_aux):
    n_subjects = len(y_pred[set_name])
    first_time = True
    for j in range(n_subjects):
        subject_id = idx_dict[set_name][j]
        if (subjects_to_show is None) or (subject_id in subjects_to_show):
            if first_time:
                ax.plot(pr_curve[set_name][j][:, 0], pr_curve[set_name][j][:, 1], 
                        label=set_name.capitalize(), alpha=alpha,
                        linewidth=1, color=color_list[set_name], zorder=10*(i+1))
                first_time = False
            else:
                ax.plot(pr_curve[set_name][j][:, 0], pr_curve[set_name][j][:, 1], alpha=alpha,
                        linewidth=1, color=color_list[set_name], zorder=10*(i+1))
# Highlight threshold=0.5
for i, set_name in enumerate(set_list_aux):
    n_subjects = len(y_pred[set_name])
    for j in range(n_subjects):
        subject_id = idx_dict[set_name][j]
        if (subjects_to_show is None) or (subject_id in subjects_to_show):
            if isinstance(thr_to_show, collections.Mapping):
                this_thr_to_show = thr_to_show[set_name][j]
                half_thr_idx = np.where(np.isclose(thr_list, this_thr_to_show))[0]
            else:
                half_thr_idx = np.where(np.isclose(thr_list, thr_to_show))[0]
            ax.scatter(pr_curve[set_name][j][half_thr_idx, 0], pr_curve[set_name][j][half_thr_idx, 1], 
                       s=markersize, c=color_list[set_name], zorder=10*(i+1))

            ax.annotate(subject_id, (pr_curve[set_name][j][half_thr_idx, 0] + text_space, pr_curve[set_name][j][half_thr_idx, 1] + text_space), 
                        fontsize=7, color='#1b2631', zorder=20)  

ax.set_title('Performance with IoU$>$%1.1f (%s)' % (iou_thr, dataset_name.upper()), fontsize=11)
ax.set_xlabel('Recall', fontsize=8.5)
ax.set_ylabel('Precision', fontsize=8.5)
ax.set_xlim([0.5, 1])
ax.set_ylim([0.5, 1])
ax.legend(loc='lower left', fontsize=8.5)
ax.tick_params(labelsize=8.5)
ax.grid()
plt.show()

## Performance: Precision-Recall curve, average per set

In [ ]:
# The mean of each set is drawn, for several thr

iou_thr = 0.3
this_thr = optimal_thr

res_thr = 0.02
start_thr = 0.1
end_thr = 0.9
n_thr = int(np.round((end_thr - start_thr) / res_thr))
thr_list = np.arange(n_thr+1) * res_thr + start_thr

# Prepare model predictions
pr_curve = {}
n_thr = len(thr_list)
for set_name in set_list:
    pr_curve[set_name] = np.zeros((n_thr, 2))   # Columns are [x: recall, y: precision]
    my_it = tqdm(enumerate(thr_list))
    for i, thr in my_it:
        my_it.set_description('Processing %s set, threshold %1.2f' % (set_name, thr))
        y_pred_thr = postprocessing.generate_mark_intervals_with_list(
            y_pred[set_name], pages[set_name], 200//8, 200, thr=thr,
            min_separation=min_separation, min_duration=min_duration, max_duration=max_duration
        )
        this_stats = [metrics.by_event_confusion(this_y, this_y_pred, iou_thr=iou_thr) 
                    for (this_y, this_y_pred) in zip(y_stamps[set_name], y_pred_thr)]
        this_recall = np.mean([m['recall'] for m in this_stats])
        this_precision = np.mean([m['precision'] for m in this_stats])
        pr_curve[set_name][i, 0] = this_recall
        pr_curve[set_name][i, 1] = this_precision

In [ ]:
# Plot settings
show_train = True
show_val = True

thr_to_show = 0.3

# ---------------------------------------------------------
fig, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=DPI)
markersize = 8
text_space = 0.01

# F1 score levels
delta = 0.01 
x_ = np.arange(1, 100) * delta 
y_ = np.arange(1, 100) * delta 
X, Y = np.meshgrid(x_, y_)
Z = 2 * X * Y / (X + Y)
CS = ax.contour(X, Y, Z, colors='k', alpha=0.3, levels=[0.6, 0.7, 0.8, 0.9])
ax.clabel(CS, fontsize=7.5, fmt='%1.2f')

set_list_aux = []
if show_train and ('alltrain' in set_list):
    set_list_aux.append('alltrain')
if show_train and ('train' in set_list):
    set_list_aux.append('train')
if show_val and ('val' in set_list):
    set_list_aux.append('val')
set_list_aux.append('test')

# Scatter plots of each subset
half_thr_idx = np.where(np.isclose(thr_list, thr_to_show))[0]
color_list = {'train':'#43a047', 'val':'#0288d1', 'test': '#c62828', 'alltrain':'#43a047'}
for i, set_name in enumerate(set_list_aux):
    ax.plot(pr_curve[set_name][:, 0], pr_curve[set_name][:, 1], 
            label=set_name.capitalize(), # markersize=markersize, marker='.', 
            linewidth=1.5, color=color_list[set_name], zorder=10*(i+1))
# Highlight threshold=0.5
for i, set_name in enumerate(set_list_aux):
    ax.scatter(pr_curve[set_name][half_thr_idx, 0], pr_curve[set_name][half_thr_idx, 1], 
               s=100, c=color_list[set_name], zorder=10*(i+1))
    ax.annotate('$\mu$=%1.3f' % thr_to_show, 
                (pr_curve[set_name][half_thr_idx, 0] + text_space*2, pr_curve[set_name][half_thr_idx, 1] + text_space), 
                fontsize=7, color='#1b2631', zorder=20)  

ax.set_title('Performance with IoU$>$%1.1f (%s)' % (iou_thr, dataset_name.upper()), fontsize=11)
ax.set_xlabel('Recall', fontsize=8.5)
ax.set_ylabel('Precision', fontsize=8.5)
ax.set_xlim([0.5, 1])
ax.set_ylim([0.5, 1])
ax.legend(loc='lower left', fontsize=8.5)
ax.tick_params(labelsize=8.5)
ax.grid()
plt.show()

## Visualization of subject

In [ ]:
thr = optimal_thr

subset_name = 'alltrain'
subject_id = 19

# -----
idx_dict = {'train': train_ids, 'val': val_ids, 'test': test_ids, 'alltrain': all_train_ids}
idx_subject = idx_dict[subset_name].index(subject_id)
this_signal = x[subset_name][idx_subject]
this_marks_1 = y[subset_name][idx_subject]
this_n2_pages = pages[subset_name][idx_subject]
this_prob = y_pred[subset_name][idx_subject]
if dataset_name == constants.MASS_NAME:
    _, this_marks_2 = dataset.get_subject_data(subject_id, which_expert=2, verbose=True)
    channel_name = 'C3-CLE'
    event_name = 'Sleep Spindle'
elif dataset_name == constants.INTA_NAME:
    channel_name = 'F4-C4'
    event_name = 'Sleep Spindle'
else:
    channel_name = 'C3-CLE'
    event_name = 'K Complex'
    
y_pred_thr = postprocessing.generate_mark_intervals(
    y_pred[subset_name][idx_subject], pages[subset_name][idx_subject], 200//8, 200, thr=thr,
    min_separation=min_separation, min_duration=min_duration, max_duration=max_duration
)
y_pred_thr_seq = data_ops.inter2seq(y_pred_thr, 0, (pages[subset_name][idx_subject].max()+1)*200*20-1)
# Now reshape into pages
this_det = data_ops.extract_pages(y_pred_thr_seq, pages[subset_name][idx_subject], 200*20, border_size=0)
    
# make a color map of fixed colors
cmap = colors.ListedColormap(['white', '#c62828'])

def plot_page(page_idx):
    if dataset_name == constants.MASS_NAME:
        fig = plt.figure(figsize=(15, 4), dpi=DPI)
        gs = gridspec.GridSpec(3, 1, height_ratios=[4, 1, 1])
    else:
        fig = plt.figure(figsize=(15, 3), dpi=DPI)
        gs = gridspec.GridSpec(2, 1, height_ratios=[4, 1])
    
    page_idx = page_idx - 1
    
    segment_signal = this_signal[page_idx, :]
    segment_marks_1 = this_marks_1[page_idx, :]
    segment_prob = this_prob[page_idx, :]
    segment_det = this_det[page_idx, :]
    time_axis = np.arange(this_signal.shape[1]) / dataset.fs
    
    if dataset_name == constants.MASS_NAME:
        segment_marks_2 = this_marks_2[page_idx, :]
    
    gs_idx = 0
    
    # Signal
    y_max = 10
    ax = fig.add_subplot(gs[gs_idx])
    gs_idx = gs_idx + 1
    ax.plot(time_axis, segment_signal, linewidth=1, color='#455a64', label='EEG %s' % channel_name)
    ax.fill_between(time_axis, y_max * segment_marks_1, -y_max * segment_marks_1, facecolor='#c62828', alpha=0.3, label=event_name)
    ax.set_yticks([])
    ax.set_xlim([0, 20])
    ax.set_ylim([-y_max, y_max])
    ax.set_title('Subject %d (%s-%s). Page in record: %d (intervals of 0.5s are shown).' 
                 % (subject_id, dataset_name.upper(), subset_name.capitalize(), this_n2_pages[page_idx]), fontsize=10)
    ax.set_xticks([0, 5, 10, 15, 20])
    ax.set_xticks(np.arange(0, 20, 0.5), minor=True)
    ax.grid(b=True, axis='x', which='minor')
    lg = ax.legend(loc='upper right', fontsize=8.5)
    # lh = lg.legendHandles[1]
    # lh.set_facecolor(lh.get_facecolor())
    # lh.set_alpha(1.0)
    ax.tick_params(labelsize=8.5)
    
    # Neural net
    delta_y = 0.1
    time_axis_short = time_axis[::8]
    ax = fig.add_subplot(gs[gs_idx])
    gs_idx = gs_idx + 1
    ax.plot(time_axis_short, segment_prob, color='#c62828', linewidth=1.5, zorder=7)
    ax.fill_between(time_axis, (1+delta_y)*segment_det, (-delta_y)*segment_det, facecolor='grey', zorder=6)
    ax.set_xticks([])
    ax.set_ylim([-delta_y, 1+delta_y])
    ax.set_xlim([0, 20])
    ax.set_yticks([0, 1])
    ax.set_yticks([thr], minor=True)
    ax.grid(b=True, axis='y', linewidth=1.5, which='minor', zorder=5)
    ax.tick_params(labelsize=8.5)
    ax.set_title('Model probability prediction (%1.2f threshold and postprocessed detections are shown)' % thr, fontsize=10)
    
    if dataset_name == constants.MASS_NAME:
        ax = fig.add_subplot(gs[gs_idx])
        ax.imshow(segment_marks_2[np.newaxis, :], interpolation=None, aspect='auto', cmap=cmap)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title('Second expert, not used for training (red indicates event)', fontsize=10)
    
    plt.tight_layout()
    plt.show()

In [ ]:
widgets.interact(
    lambda page_idx: plot_page(page_idx),
    page_idx=widgets.IntSlider(min=1,max=this_n2_pages.shape[0],step=1,value=1, continuous_update=False));

# Error analysis

### Postprocessing sensitivity

In [ ]:
chosen_set_error = 'test'
thr = optimal_thr

y_thr = y_stamps[chosen_set_error]

# Prepare model predictions
n_subjects = len(y_thr)
pred_stamps = []
for i in range(n_subjects):
    print('Processing subject %d/%d' % (i+1, n_subjects), flush=True)
    # Binarize
    this_y_pred_thr = (y_pred[chosen_set_error][i] >= thr).astype(np.int32)
    # Transform to intervals
    this_y_pred_thr = data_ops.seq2inter_with_pages(
        this_y_pred_thr, pages[chosen_set_error][i]
    )
    pred_stamps.append(this_y_pred_thr)
fs_pred = 200 // 8 
fs_real = 200
print('Done')

In [ ]:
# ------------------------- Compute spacing ----------------------
combine_thr = 0.3

spacing = []
spacing_expert = []
for i in range(n_subjects):
    print('Spacing for subject %d/%d' % (i+1, n_subjects), flush=True)
    this_spacing = (pred_stamps[i][1:, 0] - pred_stamps[i][:-1, 1]) / fs_pred
    this_spacing_expert = (y_stamps[chosen_set_error][i][1:, 0] - y_stamps[chosen_set_error][i][:-1, 1]) / fs_real
    # Only consider for this analysis, spacing less than one second
    this_spacing = this_spacing[this_spacing < 1]
    this_spacing_expert = this_spacing_expert[this_spacing_expert < 1]
    spacing.append(this_spacing)
    spacing_expert.append(this_spacing_expert)
print('Done')

fig, ax = plt.subplots(n_subjects, 2, figsize=(8, 1*n_subjects), dpi=DPI, sharex=True, sharey=False)
ax[0, 0].set_title('Spacing between nearby expert marks', fontsize=10)
ax[0, 1].set_title('Spacing between nearby detections', fontsize=10)
y_max = 0
idx_dict = {'train': train_ids, 'val': val_ids, 'test': test_ids}
for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    # Real
    n_real, _, _ = ax[i, 0].hist(spacing_expert[i], bins=[k*0.1 for k in range(11)], color='#455a64', label='S%02d (%s)' % (subject_idx, chosen_set_error.capitalize()))
    ax[i, 0].set_xticks([k*0.2 for k in range(6)])
    ax[i, 0].tick_params(labelsize=8.5)
    ax[i, 0].legend(fontsize=8.5)
    # Predicted
    n_pred, _, _ = ax[i, 1].hist(spacing[i], bins=[k*0.1 for k in range(11)], color='#c62828', label='S%02d (%s)' % (subject_idx, chosen_set_error.capitalize()))
    ax[i, 1].set_xticks([k*0.2 for k in range(6)])
    ax[i, 1].tick_params(labelsize=8.5)
    ax[i, 1].legend(fontsize=8.5)
    max_y = max(np.max(n_real), np.max(n_pred))
    ax[i, 0].plot([combine_thr, combine_thr], [0, max_y], '--', color='k', linewidth=2, alpha=0.6)
    ax[i, 1].plot([combine_thr, combine_thr], [0, max_y], '--', color='k', linewidth=2, alpha=0.6)
    # y_max = max(max_y, y_max)
# for i in range(n_subjects):
#     ax[i, 0].plot([combine_thr, combine_thr], [0, y_max], '--', color='k', linewidth=2, alpha=0.6)
#     ax[i, 1].plot([combine_thr, combine_thr], [0, y_max], '--', color='k', linewidth=2, alpha=0.6)
ax[-1, 0].set_xlabel('Spacing [s]', fontsize=8.5)
ax[-1, 1].set_xlabel('Spacing [s]', fontsize=8.5)
plt.show()

In [ ]:
# -------------- Durations

if dataset_name == constants.MASSK_NAME:
    postprocess_predicted = False
else:
    postprocess_predicted = True

durations = []
durations_expert = []
for i in range(n_subjects):
    print('Durations for subject %d/%d' % (i+1, n_subjects), flush=True)
    # First, combine close marks
    if dataset_name == constants.MASSK_NAME:
        this_pred_stamps = pred_stamps[i]
    else:
        this_pred_stamps = postprocessing.combine_close_marks(pred_stamps[i], fs_pred, combine_thr)
    # Now compute durations
    this_durations = (this_pred_stamps[:, 1] - this_pred_stamps[:, 0]) / fs_pred
    this_durations_expert = (y_stamps[chosen_set_error][i][:, 1] - y_stamps[chosen_set_error][i][:, 0]) / fs_real
    # Only consider for this analysis, spacing less than one second
    durations.append(this_durations)
    durations_expert.append(this_durations_expert)
print('Done')

fig, ax = plt.subplots(n_subjects, 2, figsize=(8, 1*n_subjects), dpi=DPI, sharex=True, sharey=False)
ax[0, 0].set_title('Duration of expert marks', fontsize=10)
ax[0, 1].set_title('Duration of detections', fontsize=10)
idx_dict = {'train': train_ids, 'val': val_ids, 'test': test_ids}
y_max = 0
min_duration = 0.3  
for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    # Real
    n_real, _, _ = ax[i, 0].hist(durations_expert[i], color='#455a64', label='S%02d (%s)' % (subject_idx, chosen_set_error.capitalize()))
    ax[i, 0].tick_params(labelsize=8.5)
    ax[i, 0].legend(fontsize=8.5)
    # Predicted
    n_pred, _, _ = ax[i, 1].hist(durations[i], color='#c62828', label='S%02d (%s)' % (subject_idx, chosen_set_error.capitalize()))
    ax[i, 1].tick_params(labelsize=8.5)
    ax[i, 1].legend(fontsize=8.5)
    max_y = max(np.max(n_real), np.max(n_pred))
    ax[i, 0].plot([min_duration, min_duration], [0, max_y], '--', color='k', linewidth=2, alpha=0.6)
    ax[i, 1].plot([min_duration, min_duration], [0, max_y], '--', color='k', linewidth=2, alpha=0.6)
    y_max = max(max_y, y_max)
    print(durations[i].max())
    
#for i in range(n_subjects):
#    ax[i, 0].plot([min_duration, min_duration], [0, y_max], '--', color='k', linewidth=2, alpha=0.6)
#    ax[i, 1].plot([min_duration, min_duration], [0, y_max], '--', color='k', linewidth=2, alpha=0.6)

ax[-1, 0].set_xlabel('Duration [s]', fontsize=8.5)
ax[-1, 1].set_xlabel('Duration [s]', fontsize=8.5)
plt.show()

### Location of Falses

In [ ]:
if dataset_name == constants.MASSK_NAME:
    min_separation = 0
    min_duration = 0.3
    max_duration = 4.0
else:
    min_separation = 0.3
    min_duration = 0.2
    max_duration = 4.0

iou_array = []
idx_array = []
y_pred_thr = []
for i in range(n_subjects):
    print('Processing subject %d/%d' % (i+1, n_subjects), flush=True)
    events = y_stamps[chosen_set_error][i]
    detections = postprocessing.generate_mark_intervals(
        y_pred[chosen_set_error][i], pages[chosen_set_error][i], 200//8, 200, thr=thr, 
        min_separation=min_separation, min_duration=min_duration, max_duration=max_duration)
    print(events.shape, detections.shape)
    this_iou_array, this_idx_array = metrics.matching(events, detections)
    iou_array.append(this_iou_array)
    idx_array.append(this_idx_array)
    y_pred_thr.append(detections)
print('Done')

In [ ]:
# --- False negatives
iou_thr = 0.3

fn_center = []
for i in range(n_subjects):
    idx_fn = iou_array[i] < iou_thr
    fn_stamps = y_stamps[chosen_set_error][i][idx_fn]
    this_fn_center = np.mean(fn_stamps, axis=1).astype(np.int32)
    fn_center.append(this_fn_center)
    
# --- False positives
fp_center = []
for i in range(n_subjects):
    # matched detections:
    idx_fp_1 = iou_array[i] < iou_thr
    idx_fp_1 = idx_array[i][idx_fp_1]
    idx_fp_1 = [idx for idx in idx_fp_1 if idx != -1]
    # Unmatched events
    n_detections = y_pred_thr[i].shape[0]
    idx_fp_2 = [idx for idx in range(n_detections) if (idx not in idx_array[i])]
    idx_fp = idx_fp_1 + idx_fp_2
    fp_stamps = y_pred_thr[i][idx_fp]
    this_fp_center = np.mean(fp_stamps, axis=1).astype(np.int32)
    fp_center.append(this_fp_center)

    
# --- Histogram of IoU values across real events
fig, ax = plt.subplots(n_subjects, 1, figsize=(3.5, 1*n_subjects), dpi=DPI, sharex=True, sharey=False)
ax[0].set_title('IoU values on expert marks', fontsize=10)
idx_dict = {'train': train_ids, 'val': val_ids, 'test': test_ids}
for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    ax[i].hist(iou_array[i], color='#455a64', label='S%02d (%s)' % (subject_idx, chosen_set_error.capitalize()), bins=[0.05*i for i in range(21)])
    ax[i].tick_params(labelsize=8.5)
    ax[i].legend(fontsize=8.5, loc='upper center')
    ax[i].set_xticks([0.2*i for i in range(6)])
ax[-1].set_xlabel('Time within page [s]', fontsize=8.5)
plt.show()
    
# --- Location in page
fn_loc_page = [np.mod(this_fn_center, 20*fs_real) for this_fn_center in fn_center]

fig, ax = plt.subplots(n_subjects, 2, figsize=(8, 1*n_subjects), dpi=DPI, sharex=True, sharey=False)
ax[0, 0].set_title('Location of expert marks with IoU < %1.2f' % (iou_thr), fontsize=11)

idx_dict = {'train': train_ids, 'val': val_ids, 'test': test_ids}
for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    ax[i, 0].hist(fn_loc_page[i] / fs_real, bins=[i*4 for i in range(6)], color='#455a64', label='S%02d (%s)' % (subject_idx, chosen_set_error.capitalize()))
    ax[i, 0].tick_params(labelsize=8.5)
    ax[i, 0].legend(fontsize=8.5, loc='upper center')
ax[-1, 0].set_xticks([i*4 for i in range(6)])
ax[-1, 0].set_xlabel('Time within page [s]', fontsize=8.5)

fp_loc_page = [np.mod(this_fp_center, 20*fs_real) for this_fp_center in fp_center]

ax[0, 1].set_title('Location of detections with IoU < %1.2f' % (iou_thr), fontsize=10)

idx_dict = {'train': train_ids, 'val': val_ids, 'test': test_ids}
for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    ax[i, 1].hist(fp_loc_page[i] / fs_real, bins=[i*4 for i in range(6)], color='#c62828', label='S%02d (%s)' % (subject_idx, chosen_set_error.capitalize()))
    ax[i, 1].tick_params(labelsize=8.5)
    ax[i, 1].legend(fontsize=8.5, loc='upper center')
ax[-1, 1].set_xticks([i*4 for i in range(6)])
ax[-1, 1].set_xlabel('Time within page [s]', fontsize=8.5)

plt.show()

# --- Location in register

fn_loc_register = [np.floor_divide(this_fn_center, 20*fs_real) for this_fn_center in fn_center]
max_page_array = [np.max(pages[chosen_set_error][i]) for i in range(n_subjects)]

max_of_all = np.max(max_page_array)

fig, ax = plt.subplots(n_subjects, 2, figsize=(8, 1*n_subjects), dpi=DPI, sharex=True, sharey=False)
ax[0, 0].set_title('Location in register of expert marks with IoU < %1.1f' % (iou_thr), fontsize=10)

idx_dict = {'train': train_ids, 'val': val_ids, 'test': test_ids}
for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    ax[i, 0].hist(fn_loc_register[i], color='#455a64', label='S%02d (%s)' % (subject_idx, chosen_set_error.capitalize()))
    ax[i, 0].tick_params(labelsize=8.5)
    ax[i, 0].legend(fontsize=8.5)
ax[-1, 0].set_xlim([0, max_of_all+10])
ax[-1, 0].set_xlabel('Page within register', fontsize=8.5)

fp_loc_register = [np.floor_divide(this_fp_center, 20*fs_real) for this_fp_center in fp_center]
max_page_array = [np.max(pages[chosen_set_error][i]) for i in range(n_subjects)]

max_of_all = np.max(max_page_array)

ax[0, 1].set_title('Location in register of detections with IoU < %1.1f' % (iou_thr), fontsize=11)

idx_dict = {'train': train_ids, 'val': val_ids, 'test': test_ids}
for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    ax[i, 1].hist(fp_loc_register[i], color='#c62828', label='S%02d (%s)' % (subject_idx, chosen_set_error.capitalize()))
    ax[i, 1].tick_params(labelsize=8.5)
    ax[i, 1].legend(fontsize=8.5)
ax[-1, 1].set_xlim([0, max_of_all+10])
ax[-1, 1].set_xlabel('Page within register', fontsize=8.5)

plt.show()

# ----- N2 pages

fig, ax = plt.subplots(n_subjects, 1, figsize=(3.5, 1*n_subjects), dpi=DPI, sharex=True, sharey=True)
ax[0].set_title('Location in register of N2 pages (%s)' % (chosen_set_error.capitalize()), fontsize=10)
idx_dict = {'train': train_ids, 'val': val_ids, 'test': test_ids}
for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    ax[i].hist(pages[chosen_set_error][i], color='#455a64', label='S%02d (%s)' % (subject_idx, chosen_set_error.capitalize()))
    ax[i].tick_params(labelsize=8.5)
    ax[i].legend(fontsize=8.5)
ax[-1].set_xlim([0, max_of_all+10])
ax[-1].set_xlabel('Page within register', fontsize=8.5)

plt.show()

### IoU vs Duration

In [ ]:
# --- Scatter of IoU values and duration of real and detected events
alpha = 0.2
markersize=10

fig, ax = plt.subplots(n_subjects, 2, figsize=(7, 1*n_subjects), dpi=DPI, sharex=True, sharey=True)
ax[0, 0].set_title('IoU vs duration of expert marks', fontsize=10)
idx_dict = {'train': train_ids, 'val': val_ids, 'test': test_ids}
for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    # Now compute durations
    this_durations_expert = (y_stamps[chosen_set_error][i][:, 1] - y_stamps[chosen_set_error][i][:, 0]) / fs_real
    ax[i, 0].scatter(this_durations_expert, iou_array[i], label='S%02d (%s)' % (subject_idx, chosen_set_error.capitalize()), color='#455a64', alpha=alpha, s=markersize)
    ax[i, 0].tick_params(labelsize=8.5)
    lg = ax[i, 0].legend(fontsize=7, loc='upper right')
    for lh in lg.legendHandles:
        lh.set_facecolor(lh.get_facecolor())
        lh.set_edgecolor(lh.get_edgecolor())
        lh.set_alpha(1.0)
    ax[i, 0].set_ylabel('IoU', fontsize=8.5)
ax[-1, 0].set_xlabel('Duration [s]', fontsize=8.5)

ax[0, 1].set_title('IoU vs duration of detections', fontsize=10)
for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    # Now compute durations for IoU > 0 
    idx_valid = (idx_array[i] > -1)
    this_iou_1 = iou_array[i][idx_valid]
    y_pred_stamps = y_pred_thr[i][idx_array[i]][idx_valid]
    this_durations_1 = (y_pred_stamps[:, 1] - y_pred_stamps[:, 0]) / fs_real
    # Now durations for IoU = 0
    n_detections = y_pred_thr[i].shape[0]
    idx_fp_2 = [idx for idx in range(n_detections) if (idx not in idx_array[i])]
    y_pred_stamps = y_pred_thr[i][idx_fp_2]
    this_durations_2 = (y_pred_stamps[:, 1] - y_pred_stamps[:, 0]) / fs_real
    this_iou_2 = np.zeros(this_durations_2.shape[0])
    # Concatenation
    this_durations = np.concatenate([this_durations_1, this_durations_2])
    this_iou = np.concatenate([this_iou_1, this_iou_2])
    ax[i, 1].scatter(this_durations, this_iou, label='S%02d (%s)' % (subject_idx, chosen_set_error.capitalize()), color='#c62828', alpha=alpha, s=markersize)
    ax[i, 1].tick_params(labelsize=8.5)
    lg = ax[i, 1].legend(fontsize=7, loc='upper right')
    for lh in lg.legendHandles:
        lh.set_facecolor(lh.get_facecolor())
        lh.set_edgecolor(lh.get_edgecolor())
        lh.set_alpha(1.0)
    # ax[i, 1].set_ylabel('IoU', fontsize=8.5)
ax[-1, 1].set_xlabel('Duration [s]', fontsize=8.5)

plt.show()


# --- Histogram of duration for IoU == 0
fig, ax = plt.subplots(n_subjects, 2, figsize=(8, 1*n_subjects), dpi=DPI, sharex=True, sharey=False)
ax[0, 0].set_title('Duration of unpaired expert marks', fontsize=10)
idx_dict = {'train': train_ids, 'val': val_ids, 'test': test_ids}
for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    # Now compute durations
    this_durations_expert = (y_stamps[chosen_set_error][i][:, 1] - y_stamps[chosen_set_error][i][:, 0]) / fs_real
    idx_zero = (iou_array[i] == 0)
    this_durations_expert_fn = this_durations_expert[idx_zero]
    ax[i, 0].hist(this_durations_expert_fn, label='S%02d (%s)' % (subject_idx, chosen_set_error.capitalize()), color='#455a64')
    ax[i, 0].tick_params(labelsize=8.5)
    lg = ax[i, 0].legend(fontsize=8.5, loc='upper right')
    
ax[-1, 0].set_xlabel('Duration [s]', fontsize=8.5)

ax[0, 1].set_title('Duration of unpaired detections', fontsize=10)
for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    n_detections = y_pred_thr[i].shape[0]
    idx_fp_2 = [idx for idx in range(n_detections) if (idx not in idx_array[i])]
    y_pred_stamps = y_pred_thr[i][idx_fp_2]
    this_durations_fp = (y_pred_stamps[:, 1] - y_pred_stamps[:, 0]) / fs_real
    ax[i, 1].hist(this_durations_fp, label='S%02d (%s)' % (subject_idx, chosen_set_error.capitalize()), color='#c62828')
    ax[i, 1].tick_params(labelsize=8.5)
    ax[i, 1].legend(fontsize=8.5, loc='upper right')
ax[-1, 1].set_xlabel('Duration [s]', fontsize=8.5)

plt.show()


### Duration of events and matched detections

In [ ]:
alpha = 0.2

fig, ax = plt.subplots(2, 2, figsize=(5, 5), dpi=DPI, sharex=False, sharey=False)
# plt.suptitle('Duration of matched events (%s)' % (chosen_set_error.capitalize()), fontsize=11)
idx_dict = {'train': train_ids, 'val': val_ids, 'test': test_ids}
for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    idx_valid = (idx_array[i] > -1)
    # Now compute durations for real
    this_durations_expert = (y_stamps[chosen_set_error][i][:, 1] - y_stamps[chosen_set_error][i][:, 0]) / fs_real
    this_durations_expert = this_durations_expert[idx_valid]
    # Now compute durations for matched detections
    y_pred_stamps = y_pred_thr[i][idx_array[i]][idx_valid]
    this_durations_det = (y_pred_stamps[:, 1] - y_pred_stamps[:, 0]) / fs_real
    if i<2:
        ax[0, i].scatter(this_durations_expert, this_durations_det, label='S%02d' % subject_idx, color='#455a64', alpha=alpha)
        ax[0, i].tick_params(labelsize=8.5)
        lg = ax[0, i].legend(fontsize=7)
        for lh in lg.legendHandles:
            lh.set_facecolor(lh.get_facecolor())
            lh.set_edgecolor(lh.get_edgecolor())
            lh.set_alpha(1.0)
        max_dur = max(this_durations_expert.max(), this_durations_det.max())
        ax[0, i].set_xlim([0, max_dur + 0.1])
        ax[0, i].set_ylim([0, max_dur + 0.1])
        if i == 0 :
            ax[0, i].set_ylabel('Detected duration [s]', fontsize=8.5)
        # ax[0, i].set_xlabel('Duration Real [s]', fontsize=8.5)
        ax[0, i].plot([0.3, 2.0], [0.3, 2.0], '--k', alpha=0.6)
    else:
        ax[1, i-2].scatter(this_durations_expert, this_durations_det, label='S%02d' % subject_idx, color='#455a64', alpha=alpha)
        ax[1, i-2].tick_params(labelsize=8.5)
        lg = ax[1, i-2].legend(fontsize=7)
        for lh in lg.legendHandles:
            lh.set_facecolor(lh.get_facecolor())
            lh.set_edgecolor(lh.get_edgecolor())
            lh.set_alpha(1.0)
        max_dur = max(this_durations_expert.max(), this_durations_det.max())
        ax[1, i-2].set_xlim([0, max_dur + 0.1])
        ax[1, i-2].set_ylim([0, max_dur + 0.1])
        if i == 2:
            ax[1, i-2].set_ylabel('Detected duration [s]', fontsize=8.5)
        ax[1, i-2].set_xlabel('Real duration [s]', fontsize=8.5)
        ax[1, i-2].plot([0.3, 2.0], [0.3, 2.0], '--k', alpha=0.6)

plt.tight_layout()
plt.show()

### Predicted probability to matched and unmatched events

In [ ]:
# Prepare probabilities
whole_y_proba = []
for i in range(n_subjects):
    this_proba = y_pred[chosen_set_error][i]
    this_pages = pages[chosen_set_error][i]
    page_size = this_proba.shape[1]
    max_page = np.max(this_pages)
    max_size = (max_page + 1) * page_size
    whole_y_proba.append(np.zeros(max_size, dtype=np.float32))
    for k, page in enumerate(this_pages):
        sample_start = page * page_size
        sample_end = (page + 1) * page_size
        whole_y_proba[i][sample_start:sample_end] = this_proba[k, :]

In [ ]:
# Matching probabilities (Candidates for TP according to IoU)
# It is expected that they have more than 0.5 since that is the threshold used for detection
print('Processing probability for matched events', flush=True)
matching_proba = []
for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    # In this case, we look for the valid detections stamps 
    idx_valid = (idx_array[i] > -1)
    y_pred_stamps = y_pred_thr[i][idx_array[i]][idx_valid]
    # And now we need to get the probabilities inside those stamps
    # First, we downsample the stamps
    y_pred_stamps = y_pred_stamps // 8
    # Now, we get the mean probability between stamps
    mean_proba_list = []
    for stamp in y_pred_stamps:
        proba_inside = whole_y_proba[i][stamp[0]:(stamp[1]+1)]
        mean_proba_inside = proba_inside.mean()
        mean_proba_list.append(mean_proba_inside)
    mean_proba_list = np.array(mean_proba_list)
    print('Mean probability for S%02d: %1.4f' % (subject_idx, mean_proba_list.mean()))
    matching_proba.append(mean_proba_list)
    
# Unmatched real events (FN)
# It is expected that they have less than 0.5 since they were missed by the tresholding
print('Processing probability for unmatched real events', flush=True)
unmatching_fn_proba = []
for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    # In this case, we look for the missed real events 
    idx_valid = (idx_array[i] == -1)
    fn_stamps = y_stamps[chosen_set_error][i][idx_valid]
    # And now we need to get the probabilities inside those stamps
    # First, we downsample the stamps
    fn_stamps = fn_stamps // 8
    # Now, we get the mean probability between stamps
    mean_proba_list = []
    for stamp in fn_stamps:
        proba_inside = whole_y_proba[i][stamp[0]:(stamp[1]+1)]
        mean_proba_inside = proba_inside.mean()
        mean_proba_list.append(mean_proba_inside)
    mean_proba_list = np.array(mean_proba_list)
    print('Mean probability for S%02d: %1.4f' % (subject_idx, mean_proba_list.mean()))
    unmatching_fn_proba.append(mean_proba_list)
    
# Unmatched detected events (FP)
# It is expected that they have more than 0.5 since they were detected
print('Processing probability for unmatched detected events', flush=True)
unmatching_fp_proba = []
for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    # In this case, we look for detected events with no match 
    n_detections = y_pred_thr[i].shape[0]
    idx_valid = [idx for idx in range(n_detections) if (idx not in idx_array[i])]
    fp_stamps = y_pred_thr[i][idx_valid]
    # And now we need to get the probabilities inside those stamps
    # First, we downsample the stamps
    fp_stamps = fp_stamps // 8
    # Now, we get the mean probability between stamps
    mean_proba_list = []
    for stamp in fp_stamps:
        proba_inside = whole_y_proba[i][stamp[0]:(stamp[1]+1)]
        mean_proba_inside = proba_inside.mean()
        mean_proba_list.append(mean_proba_inside)
    mean_proba_list = np.array(mean_proba_list)
    print('Mean probability for S%02d: %1.4f' % (subject_idx, mean_proba_list.mean()))
    unmatching_fp_proba.append(mean_proba_list)
    

# Probability for real events
print('Processing probability for real events', flush=True)
real_proba = []
for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    # And now we need to get the probabilities inside those stamps
    # First, we downsample the stamps
    y_pred_stamps = y_stamps[chosen_set_error][i] // 8
    # Now, we get the mean probability between stamps
    mean_proba_list = []
    for stamp in y_pred_stamps:
        proba_inside = whole_y_proba[i][stamp[0]:(stamp[1]+1)]
        mean_proba_inside = proba_inside.mean()
        mean_proba_list.append(mean_proba_inside)
    mean_proba_list = np.array(mean_proba_list)
    print('Mean probability for S%02d: %1.4f' % (subject_idx, mean_proba_list.mean()))
    real_proba.append(mean_proba_list)

In [ ]:
# --- Histogram of probabilities

fig, ax = plt.subplots(n_subjects, 3, figsize=(14, 3*n_subjects), dpi=100, sharex=True)

ax[0, 0].set_title('Model output on matched events (%s)' % (chosen_set_error.capitalize()), fontsize=11)
idx_dict = {'train': train_ids, 'val': val_ids, 'test': test_ids}
for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    n, _, _ = ax[i, 0].hist(matching_proba[i], label='S%02d' % subject_idx, color='#43a047')
    max_n = np.max(n)
    ax[i, 0].plot([thr, thr], [0, max_n], '--', color='k', linewidth=1.5, alpha=0.6, label='thr')
    ax[i, 0].tick_params(labelsize=8.5)
    ax[i, 0].legend(fontsize=8.5)
ax[-1, 0].set_xlim([0, 1])
ax[-1, 0].set_xlabel('Probability', fontsize=8.5)

ax[0, 1].set_title('Probability on unmatched real events (%s)' % (chosen_set_error.capitalize()), fontsize=11)
for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    n, _, _ = ax[i, 1].hist(unmatching_fn_proba[i], label='S%02d' % subject_idx, color='#455a64')
    max_n = np.max(n)
    ax[i, 1].plot([thr, thr], [0, max_n], '--', color='k', linewidth=1.5, alpha=0.6, label='thr')
    ax[i, 1].tick_params(labelsize=8.5)
    ax[i, 1].legend(fontsize=8.5)
ax[-1, 1].set_xlim([0, 1])
ax[-1, 1].set_xlabel('Probability', fontsize=8.5)

ax[0, 2].set_title('Probability on unmatched detections (%s)' % (chosen_set_error.capitalize()), fontsize=11)
for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    n, _, _ = ax[i, 2].hist(unmatching_fp_proba[i], label='S%02d' % subject_idx, color='#c62828')
    max_n = np.max(n)
    ax[i, 2].plot([thr, thr], [0, max_n], '--', color='k', linewidth=1.5, alpha=0.6, label='thr')
    ax[i, 2].tick_params(labelsize=8.5)
    ax[i, 2].legend(fontsize=8.5)
ax[-1, 2].set_xlim([0, 1])
ax[-1, 2].set_xlabel('Probability', fontsize=8.5)

plt.show()

In [ ]:
# --- Histogram of probabilities

n_points = 100
x_points =np.arange(n_points + 1) / n_points

fig, ax = plt.subplots(n_subjects, 1, figsize=(6, 5*n_subjects), dpi=100, sharex=True)

for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    ax[i].set_title('Predicted probability for S%02d (%s)' % (subject_idx, chosen_set_error.capitalize()), fontsize=11)


idx_dict = {'train': train_ids, 'val': val_ids, 'test': test_ids}
max_n = 0
for i in range(n_subjects):
    
    n, _, _ = ax[i].hist(
        matching_proba[i], label='Matched real', color='#43a047', alpha=0.4, density=True,
        bins = [i*0.05 for i in range(21)])
    kernel = gaussian_kde(matching_proba[i])
    y_kde = kernel(x_points)
    ax[i].plot(x_points, y_kde, color='#43a047', linewidth=2)
    max_n = max(np.max(n), max_n)
    ax[i].tick_params(labelsize=8.5)
ax[-1].set_xlim([0, 1])
ax[-1].set_xlabel('Probability', fontsize=8.5)

# for i in range(n_subjects):
#     subject_idx = idx_dict[chosen_set_error][i]
#     n, _, _ = ax[i].hist(
#         unmatching_fn_proba[i], label='Unmatched real', color='#455a64', alpha=0.4, density=True,
#         bins = [i*0.05 for i in range(21)])
#     kernel = gaussian_kde(unmatching_fn_proba[i])
#     y_kde = kernel(x_points)
#     ax[i].plot(x_points, y_kde, color='#455a64', linewidth=2)
#     max_n = max(np.max(n), max_n)

for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    n, _, _ = ax[i].hist(
        unmatching_fp_proba[i], label='Unmatched detection', color='#c62828', alpha=0.4, density=True,
        bins = [i*0.05 for i in range(21)])
    kernel = gaussian_kde(unmatching_fp_proba[i])
    y_kde = kernel(x_points)
    ax[i].plot(x_points, y_kde, color='#c62828', linewidth=2)
    max_n = max(np.max(n), max_n)
    ax[i].plot([thr, thr], [0, max_n], '--', color='k', linewidth=1.5, alpha=0.6, label='thr')
    ax[i].legend(fontsize=8.5, loc='upper left')
    ax[i].set_yticks([])
plt.show()

In [ ]:
# --- Histogram of probabilities

n_points = 100
x_points =np.arange(n_points + 1) / n_points

fig, ax = plt.subplots(2, 2, figsize=(6, 5), dpi=DPI, sharex=True)

for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    if i < 2:
        ax[0, i].set_title('Model Output for S%02d (%s)' % (subject_idx, chosen_set_error.capitalize()), fontsize=10)
    else:
        ax[1, i-2].set_title('Model Output for S%02d (%s)' % (subject_idx, chosen_set_error.capitalize()), fontsize=10)


idx_dict = {'train': train_ids, 'val': val_ids, 'test': test_ids}
max_n = 0
for i in range(n_subjects):
    if i < 2:
        n, _, _ = ax[0, i].hist(
            real_proba[i], label='Expert marks', color='#43a047', alpha=0.4, density=True,
            bins = [k*0.05 for k in range(21)])
    else:
        n, _, _ = ax[1, i-2].hist(
            real_proba[i], label='Real events\n(Expert marks)', color='#43a047', alpha=0.4, density=True,
            bins = [k*0.05 for k in range(21)])
    kernel = gaussian_kde(real_proba[i])
    y_kde = kernel(x_points)
    if i<2:
        ax[0, i].plot(x_points, y_kde, color='#43a047', linewidth=2)
        ax[0, i].tick_params(labelsize=8.5)
        ax[0, i].set_xlim([0, 1])
    else:
        ax[1, i-2].plot(x_points, y_kde, color='#43a047', linewidth=2)
        ax[1, i-2].tick_params(labelsize=8.5)
        ax[1, i-2].set_xlim([0, 1])
    max_n = max(np.max(n), max_n)
    

# ax[-1].set_xlabel('Probability', fontsize=8.5)

for i in range(n_subjects):
    subject_idx = idx_dict[chosen_set_error][i]
    kernel = gaussian_kde(unmatching_fp_proba[i])
    y_kde = kernel(x_points)
    max_n = max(np.max(n), max_n)
    if i<2:
        n, _, _ = ax[0, i].hist(
            unmatching_fp_proba[i], label='Unpaired detections ($\mu$=0.1)', color='#c62828', alpha=0.4, density=True,
            bins = [k*0.05 for k in range(21)])
        ax[0, i].plot(x_points, y_kde, color='#c62828', linewidth=2)
        # ax[i].plot([thr, thr], [0, max_n], '--', color='k', linewidth=1.5, alpha=0.6, label='thr')
        # lg = ax[0, i].legend(fontsize=8.5, loc='upper center', bbox_to_anchor=(1.05, 0.15))
        # for lh in lg.legendHandles:
        #     lh.set_facecolor(lh.get_facecolor())
        #     lh.set_alpha(1.0)
        ax[0, i].set_yticks([])
        ax[0, i].tick_params(labelsize=8.5)
    else:
        n, _, _ = ax[1, i-2].hist(
            unmatching_fp_proba[i], label='False events\n(Unpaired detections\ngenerated with $\mu$=0.1)', color='#c62828', alpha=0.4, density=True,
            bins = [k*0.05 for k in range(21)])
        ax[1, i-2].plot(x_points, y_kde, color='#c62828', linewidth=2)
        # ax[i].plot([thr, thr], [0, max_n], '--', color='k', linewidth=1.5, alpha=0.6, label='thr')
        ax[1, i-2].set_yticks([])
        ax[1, i-2].set_xlabel('Class assignment', fontsize=8.5)
        ax[1, i-2].tick_params(labelsize=8.5)
lg = ax[1, 1].legend(fontsize=9, loc='upper left', bbox_to_anchor=(1.05, 1.35), labelspacing=3)
for lh in lg.legendHandles:
    lh.set_facecolor(lh.get_facecolor())
    lh.set_alpha(1.0)
plt.show()

In [ ]:
# --- Avg distributions over set


# --- Histogram of probabilities

n_points = 100
x_points =np.arange(n_points + 1) / n_points

fig, ax = plt.subplots(1, 1, figsize=(7, 5), dpi=100, sharex=True)

ax.set_title('Predicted probability. Average for %s set' % (chosen_set_error.capitalize()), fontsize=11)

y_kde_list = []
for i in range(n_subjects):
    kernel = gaussian_kde(matching_proba[i])
    y_kde = kernel(x_points)
    y_kde_list.append(y_kde)
y_kde_avg_real = np.stack(y_kde_list, axis=1).mean(axis=1)
ax.plot(x_points, y_kde_avg_real, color='#43a047', linewidth=2, label='Real event')
ax.fill_between(x_points, y_kde_avg_real, 0*y_kde_avg_real, color='#43a047', alpha=0.4)

y_kde_list = []
for i in range(n_subjects):
    kernel = gaussian_kde(unmatching_fp_proba[i])
    y_kde = kernel(x_points)
    y_kde_list.append(y_kde)
y_kde_avg_false = np.stack(y_kde_list, axis=1).mean(axis=1)
ax.plot(x_points, y_kde_avg_false, color='#c62828', linewidth=2, label='False event')
ax.fill_between(x_points, y_kde_avg_false, 0*y_kde_avg_false, color='#c62828', alpha=0.4)

max_y = max(np.max(y_kde_avg_real), np.max(y_kde_avg_false))

# Find optimal threshold
difference = y_kde_avg_false - y_kde_avg_real
idx_thr = np.where(np.signbit(difference))[0][0]
x_thr = x_points[idx_thr]
print('Optimal Threshold: %1.4f' % x_thr)
ax.plot([x_thr, x_thr], [0, max_y], '--', color='k', linewidth=1.5, alpha=0.6, label='Optimal Threshold')

ax.tick_params(labelsize=8.5)
ax.set_xlim([0, 1])
ax.set_ylim([0, max_y])
ax.set_yticks([])
ax.legend(fontsize=8.5, loc='upper left')
ax.set_xlabel('Probability', fontsize=8.5)
ax.set_xticks([i*0.1 for i in range(11)])

plt.show()

### Analysis of FP that are TP-E2

In [ ]:
# Find unmatched events according to E1
unmatched_stamps = []

idx_dict = {'train': train_ids, 'val': val_ids, 'test': test_ids}
for i in range(n_subjects):
    # Unmatched detections
    subject_idx = idx_dict[chosen_set_error][i]
    n_detections = y_pred_thr[i].shape[0]
    idx_fp = [idx for idx in range(n_detections) if (idx not in idx_array[i])]
    fp_stamps = y_pred_thr[i][idx_fp]
    print('\n%d / %d (%1.2f %%) unmatched detections with E1 for S%02d' % (fp_stamps.shape[0], n_detections, 100*fp_stamps.shape[0]/n_detections, subject_idx))
    unmatched_stamps.append(fp_stamps)
    # Now match with E2
    events = y2_stamps[chosen_set_error][i]
    detections = fp_stamps
    n_detections = fp_stamps.shape[0]
    this_iou_array, this_idx_array = metrics.matching(events, detections)
    matched_2 =  (this_idx_array > -1).sum()
    print('%d were matched with E2 (%1.2f%% of previously unmatched detections)' % (matched_2, 100*matched_2 / fp_stamps.shape[0]))
    # print(fp_stamps[this_idx_array[this_idx_array > -1]])